In [1]:
%config IPCompleter.use_jedi = False
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#historical
# reuters, yahoo, https://github.com/niderhoff/nlp-datasets
# current
import findspark
findspark.init()
from newsapi import NewsApiClient
import pyspark
from pyspark import SparkConf, SparkContext, SQLContext
from os import environ
import os
import pandas as pd
from pyspark.ml.feature import *
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [3]:
def download_dataset(folder_path):
    import kaggle as k

    kaggle_json_api_key = {"username":"lukemcleary","key":"fa6dddf57695be72300b394324ef669f"}
    if environ.get("KAGGLE_USERNAME") is None:
        environ["KAGGLE_USERNAME"] = kaggle_json_api_key["username"]
    if environ.get("KAGGLE_KEY") is None:
        environ["KAGGLE_KEY"] = kaggle_json_api_key["key"]
    k.api.authenticate()
    k.dataset_download_files('therohk/million-headlines', path=folder_path, unzip=True,quiet =False)    

In [4]:
folder_path = 'million-headlines'

In [5]:
if not os.path.exists(folder_path):
    download_dataset(folder_path)

In [6]:
#spark set-up
conf = (SparkConf()
        .setMaster("local")
        .setAppName("My app"))
sc = SparkContext(conf=conf)


sqlContext = SQLContext(sc)

spark = sqlContext.sparkSession.builder.appName(
    "Spark").getOrCreate()

In [7]:
f = os.listdir(folder_path)[0]
f_path = os.path.join(folder_path, f)

In [14]:
%timeit dfp = pd.read_csv(f_path)

2.21 s ± 294 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
dfp = pd.read_csv(f_path)

In [16]:
dfp.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [17]:
dfp.shape[0]

1186018

In [18]:
#spark

In [8]:
%time dfs = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load(f_path)

Wall time: 21.3 s


In [9]:
dfs = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load(f_path)

In [10]:
dfs.show(5)

+------------+--------------------+
|publish_date|       headline_text|
+------------+--------------------+
|    20030219|aba decides again...|
|    20030219|act fire witnesse...|
|    20030219|a g calls for inf...|
|    20030219|air nz staff in a...|
|    20030219|air nz strike to ...|
+------------+--------------------+
only showing top 5 rows



In [11]:
dfs.count()

1186018

In [23]:
# rows containing 'US'

In [24]:
dfp[dfp.headline_text.str.contains('fire')]

,publish_date,headline_text
1,20030219,act fire witnesses must be aware of defamation
19,20030219,bushfire victims urged to see centrelink
34,20030219,dargo fire threat expected to rise
48,20030219,firefighters contain acid spill
63,20030219,griffiths under fire over project knock back
...,...,...
1186001,20191231,sydney new years eve fireworks be replaced by ...
1186004,20191231,the fire is engulfing the house: tracey corbin
1186010,20191231,victoria bushfire mallacoota residents emergency
1186011,20191231,victoria bushfire trapping people in east gipp...


In [12]:
dfs.createOrReplaceTempView("table")

In [13]:
spark.sql(r"SELECT * FROM table where headline_text like '%fire%'").collect()

headline_text='iraqs key oil pipeline to turkey hit by fire'),
 Row(publish_date=20030816, headline_text='man charged with murder after fatal house fire'),
 Row(publish_date=20030817, headline_text='12 hurt in french fireworks explosion'),
 Row(publish_date=20030817, headline_text='fire still raging at iraqi oil export pipeline'),
 Row(publish_date=20030817, headline_text='pak fires eight birdies at suspended lpga jamie'),
 Row(publish_date=20030818, headline_text='arson suspected in house fire'),
 Row(publish_date=20030818, headline_text='fire destroys cultural centre'),
 Row(publish_date=20030818, headline_text='firefighters find body in melbourne'),
 Row(publish_date=20030818, headline_text='gunfire resounds across nigerian oil city'),
 Row(publish_date=20030818, headline_text='s korea fires on north korean boat'),
 Row(publish_date=20030818, headline_text='warehouse fire proves difficult for firefighters'),
 Row(publish_date=20030818, headline_text='wood fired up for world cup'),
 

In [14]:
%time spark.sql(r"SELECT * FROM table").where("length(headline_text)>75").collect()

Wall time: 4.45 s


[Row(publish_date=20130522, headline_text='84yo dead after crashing into sydney house 80yo passenger in serious condition'),
 Row(publish_date=20130716, headline_text='icac hears phone tap outlining sydney mayors plan to crucify general manager')]

In [28]:
%time dfp[dfp['headline_text'].str.len()>75]

Wall time: 852 ms


,publish_date,headline_text
788614,20130522,84yo dead after crashing into sydney house 80y...
803047,20130716,icac hears phone tap outlining sydney mayors p...


In [15]:
dfs.rdd.getNumPartitions()

1

In [16]:
f_path

'million-headlines\\abcnews-date-text.csv'

In [49]:
# rdd = sc.textFile(f_path)
# rdd.take(5)

In [51]:
def filter_by_len(x):
    return len(x.split(',')[1])>75


In [52]:
# %time rdd.filter(lambda x: filter_by_len(x)).collect()

Wall time: 9.08 s


['20130522,84yo dead after crashing into sydney house 80yo passenger in serious condition',
 '20130716,icac hears phone tap outlining sydney mayors plan to crucify general manager']

In [53]:
# rdd.getNumPartitions()

2

In [17]:
dfs.take(5)

[Row(publish_date=20030219, headline_text='aba decides against community broadcasting licence'),
 Row(publish_date=20030219, headline_text='act fire witnesses must be aware of defamation'),
 Row(publish_date=20030219, headline_text='a g calls for infrastructure protection summit'),
 Row(publish_date=20030219, headline_text='air nz staff in aust strike for pay rise'),
 Row(publish_date=20030219, headline_text='air nz strike to affect australian travellers')]

In [18]:
tokenizer = Tokenizer(inputCol="headline_text", outputCol = "words")

In [19]:
regextokenizer = RegexTokenizer(inputCol="headline_text", outputCol = "words", pattern="\\w")

In [20]:
countTokens = udf(lambda words: len(words), IntegerType())

In [22]:
tokenized = tokenizer.transform(dfs)


In [23]:
tokenized

DataFrame[publish_date: int, headline_text: string, words: array<string>]

In [26]:
tokenized.withColumn("tokens", countTokens(col("words"))).show()

+------------+--------------------+--------------------+------+
|publish_date|       headline_text|               words|tokens|
+------------+--------------------+--------------------+------+
|    20030219|aba decides again...|[aba, decides, ag...|     6|
|    20030219|act fire witnesse...|[act, fire, witne...|     8|
|    20030219|a g calls for inf...|[a, g, calls, for...|     7|
|    20030219|air nz staff in a...|[air, nz, staff, ...|     9|
|    20030219|air nz strike to ...|[air, nz, strike,...|     7|
|    20030219|ambitious olsson ...|[ambitious, olsso...|     5|
|    20030219|antic delighted w...|[antic, delighted...|     6|
|    20030219|aussie qualifier ...|[aussie, qualifie...|     7|
|    20030219|aust addresses un...|[aust, addresses,...|     7|
|    20030219|australia is lock...|[australia, is, l...|     7|
|    20030219|australia to cont...|[australia, to, c...|     9|
|    20030219|barca take record...|[barca, take, rec...|     8|
|    20030219|bathhouse plans m...|[bath

In [24]:
tokenized.select("headline_text", "words")\
    .withColumn("tokens", countTokens(col("words"))).show(truncate=False)

+--------------------------------------------------+-----------------------------------------------------------+------+
|headline_text                                     |words                                                      |tokens|
+--------------------------------------------------+-----------------------------------------------------------+------+
|aba decides against community broadcasting licence|[aba, decides, against, community, broadcasting, licence]  |6     |
|act fire witnesses must be aware of defamation    |[act, fire, witnesses, must, be, aware, of, defamation]    |8     |
|a g calls for infrastructure protection summit    |[a, g, calls, for, infrastructure, protection, summit]     |7     |
|air nz staff in aust strike for pay rise          |[air, nz, staff, in, aust, strike, for, pay, rise]         |9     |
|air nz strike to affect australian travellers     |[air, nz, strike, to, affect, australian, travellers]      |7     |
|ambitious olsson wins triple jump      

In [73]:
regexTokenized = regextokenizer.transform(dfs)
regexTokenized.select("headline_text", "words") \
    .withColumn("tokens", countTokens(col("words"))).show(truncate=False)

+--------------------------------------------------+------------------------+------+
|headline_text                                     |words                   |tokens|
+--------------------------------------------------+------------------------+------+
|aba decides against community broadcasting licence|[ ,  ,  ,  ,  ]         |5     |
|act fire witnesses must be aware of defamation    |[ ,  ,  ,  ,  ,  ,  ]   |7     |
|a g calls for infrastructure protection summit    |[ ,  ,  ,  ,  ,  ]      |6     |
|air nz staff in aust strike for pay rise          |[ ,  ,  ,  ,  ,  ,  ,  ]|8     |
|air nz strike to affect australian travellers     |[ ,  ,  ,  ,  ,  ]      |6     |
|ambitious olsson wins triple jump                 |[ ,  ,  ,  ]            |4     |
|antic delighted with record breaking barca        |[ ,  ,  ,  ,  ]         |5     |
|aussie qualifier stosur wastes four memphis match |[ ,  ,  ,  ,  ,  ]      |6     |
|aust addresses un security council over iraq      |[ ,  ,  ,  , 

In [74]:
regexTokenized.select("headline_text", "words") \
    .withColumn("tokens", countTokens(col("words"))).show()

+--------------------+--------------------+------+
|       headline_text|               words|tokens|
+--------------------+--------------------+------+
|aba decides again...|     [ ,  ,  ,  ,  ]|     5|
|act fire witnesse...|[ ,  ,  ,  ,  ,  ...|     7|
|a g calls for inf...|  [ ,  ,  ,  ,  ,  ]|     6|
|air nz staff in a...|[ ,  ,  ,  ,  ,  ...|     8|
|air nz strike to ...|  [ ,  ,  ,  ,  ,  ]|     6|
|ambitious olsson ...|        [ ,  ,  ,  ]|     4|
|antic delighted w...|     [ ,  ,  ,  ,  ]|     5|
|aussie qualifier ...|  [ ,  ,  ,  ,  ,  ]|     6|
|aust addresses un...|  [ ,  ,  ,  ,  ,  ]|     6|
|australia is lock...|  [ ,  ,  ,  ,  ,  ]|     6|
|australia to cont...|[ ,  ,  ,  ,  ,  ...|     8|
|barca take record...|[ ,  ,  ,  ,  ,  ...|     7|
|bathhouse plans m...|           [ ,  ,  ]|     3|
|big hopes for lau...|     [ ,  ,  ,  ,  ]|     5|
|big plan to boost...|  [ ,  ,  ,  ,  ,  ]|     6|
|blizzard buries u...|     [ ,  ,  ,  ,  ]|     5|
|brigadier dismiss...|     [ , 

In [27]:
from spark.ml.feature import Pipeline

DEBUG:py4j.java_gateway:Command to send: r
u
SparkConf
rj
e

DEBUG:py4j.java_gateway:Answer received: !ycorg.apache.spark.SparkConf
DEBUG:py4j.java_gateway:Command to send: i
org.apache.spark.SparkConf
bTrue
e

DEBUG:py4j.java_gateway:Answer received: !yro89
DEBUG:py4j.java_gateway:Command to send: c
o89
set
sspark.master
slocal
e

DEBUG:py4j.java_gateway:Answer received: !yro90
DEBUG:py4j.java_gateway:Command to send: m
d
o90
e

DEBUG:py4j.java_gateway:Answer received: !yv
DEBUG:py4j.java_gateway:Command to send: c
o89
set
sspark.app.name
sMy app
e

DEBUG:py4j.java_gateway:Answer received: !yro91
DEBUG:py4j.java_gateway:Command to send: m
d
o91
e

DEBUG:py4j.java_gateway:Answer received: !yv
DEBUG:py4j.java_gateway:Command to send: c
o89
set
sspark.executor.memory
s1g
e

DEBUG:py4j.java_gateway:Answer received: !yro92
DEBUG:py4j.java_gateway:Command to send: m
d
o92
e

DEBUG:py4j.java_gateway:Answer received: !yv
DEBUG:py4j.java_gateway:Command to send: c
o89
set
sspark.driver.extraCl

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=My app, master=local) created by __init__ at <ipython-input-6-e1df8a03791e>:5 

In [ ]:
ner_dl = NerDLModel.pretrained(‘ner_dl’)

In [ ]:
b